In [80]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [81]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
!pip install konlpy 
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [148]:
tokenizer = Tokenizer(5542)

In [183]:
import json

with open('/content/drive/MyDrive/Ybigta/신입기수 프로젝트/감성분석 모델링 추가(Bi-LSTM, Attention)/wordIndex(BiLSTM2).json') as json_file:
  word_index = json.load(json_file)
  tokenizer.word_index = word_index

from keras.models import load_model
model = load_model('/content/drive/MyDrive/Ybigta/신입기수 프로젝트/감성분석 모델링 추가(Bi-LSTM, Attention)/BiLSTM_model(sigmoid).h5')

In [184]:
okt = Okt()

In [185]:
max_len=30

In [186]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

from pykospacing import spacing


  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-idfsx8yc
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-idfsx8yc
  Created wheel for pykospacing: filename=pykospacing-0.4-cp37-none-any.whl size=2255638 sha256=f55221ca43a0e19f7d14e55f91d2a1dc6f3235524207db39e99e1cc44ed51725
  Stored in directory: /tmp/pip-ephem-wheel-cache-i25p88ll/wheels/4d/45/58/e26cb2b7f6a063d234158c6fd1e5700f6e15b99d67154340ba
Successfully built pykospacing


In [187]:
!pip install git+https://github.com/ssut/py-hanspell.git

from hanspell import spell_checker
def grammar_check(text):
  spelled_sent = spell_checker.check(text)
  hanspell_sent = spelled_sent.checked
  return hanspell_sent

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-pj52fpdp
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-pj52fpdp
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp37-none-any.whl size=4854 sha256=ce631964e7e99f87c25e119c139330fb798449e77bbe51885be73aa62bb97c28
  Stored in directory: /tmp/pip-ephem-wheel-cache-11on8je2/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


In [188]:
import re

punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&' 
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2",
                 "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 
                 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi'} 

def clean_punc(text, punct, mapping): 
  for p in mapping: 
    text = text.replace(p, mapping[p]) 
  for p in punct: 
    text = text.replace(p, f' {p} ') 
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''} 
  for s in specials: 
    text = text.replace(s, specials[s]) 
   
  return text.strip() 



def clean_text(texts): 
  corpus = [] 
  for i in range(0, len(texts)): 
    review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation 
    review = re.sub(r'\d+','', str(texts[i]))# remove number 
    review = review.lower() #lower case 
    review = re.sub(r'\s+', ' ', review) #remove extra space 
    review = re.sub(r'<[^>]+>','',review) #remove Html tags 
    review = re.sub(r'\s+', ' ', review) #remove spaces 
    review = re.sub(r"^\s+", '', review) #remove space from start 
    review = re.sub(r'\s+$', '', review) #remove space from the end 
    review = re.sub(r'♥','',review)
    review = re.sub(r'~','',review)
    review = re.sub(r'&','',review)
    review = re.sub(r'😱','',review)
    corpus.append(review) 
  return corpus



# removing emoji

def rmEmoji(text):
  review = text.encode('utf-8', 'ignore').decode('utf-8')
  return review

def rmEmoji_ascii(inputString):
  review = example.encode('ascii','ignore').decode('ascii')
  return review


In [189]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
# NLTK Data 다운
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = open('/content/drive/MyDrive/Ybigta/korean_stopwords.txt').read()


# 위의 불용어는 명사가 아닌 단어 중에서 저자가 임의로 선정한 것으로 실제 의미있는 선정 기준이 아님
stop_words=stop_words.split('\n')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [190]:
def tokenize_tagged(text):

  temp_X = okt.pos(text, norm=True, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stop_words] # 불용어 제거
  return ['/'.join(t) for t in temp_X]

In [191]:
tokenize_tagged("안녕 만나서 반가워")

['안녕/Noun', '만나다/Verb', '반갑다/Adjective']

In [192]:
def sentiment_predict(new_sentence):
  text1 = clean_punc(new_sentence, punct, punct_mapping)
  text2 = ''.join(clean_text(text1))
  text3 = spacing(text2)
  text4 = grammar_check(text3)
  text_ = tokenize_tagged(text4)

  #new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  print(text_)
  #new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  #print(new_sentence)
  encoded = tokenizer.texts_to_sequences([text_]) # 정수 인코딩
  print(encoded)
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  print(pad_new)
  score = float(model.predict(pad_new)) # 예측
  
  print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  #if(score > 0.5):
    #print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  #else:
    #print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [193]:
sentiment_predict('여기는 커피가 정말 너무 맛있어요 최고예요 직원도 친절해요')

['여기다/Verb', '커피/Noun', '가/Josa', '정말/Noun', '너무/Adverb', '맛있다/Adjective', '최고/Noun', '예/Noun', '요/Noun', '직원/Noun', '도/Josa', '친절하다/Adjective']
[[196, 7, 32, 8, 4, 10, 929, 36, 17, 18]]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0 196   7  32   8   4  10 929  36  17  18]]
99.87% 확률로 긍정 리뷰입니다.



In [194]:
sentiment_predict('더러워 직원이 불친절해요 짜증나 싫어')

['더럽다/Adjective', '직원/Noun', '이/Josa', '불친절하다/Adjective', '짜증/Noun', '나/Noun', '싫다/Adjective']
[[398, 17, 67, 601, 109, 390]]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0 398  17  67 601 109 390]]
2.41% 확률로 긍정 리뷰입니다.



In [179]:
sentiment_predict('그냥 저냥 보통이었어요 오늘 하루종일 기분이 우울해요')

['그냥/Modifier', '저/Noun', '냥/Josa', '보통/Noun', '이다/Verb', '오늘/Noun', '하루/Noun', '종일/Noun', '기분/Noun', '이/Josa', '우울하다/Adjective']
[[1014, 135, 367, 68, 317, 850, 1672, 98]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0 1014  135  367   68  317  850
  1672   98]]
45.76% 확률로 긍정 리뷰입니다.



In [180]:
sentiment_predict('맛있네요 예쁘네요 넓어요')

['맛있다/Adjective', '예쁘다/Adjective', '넓다/Adjective']
[[4, 63, 85]]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  4 63 85]]
187.72% 확률로 긍정 리뷰입니다.



In [181]:
sentiment_predict('공간이 넓어요')

['공간/Noun', '이/Josa', '넓다/Adjective']
[[61, 85]]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0 61 85]]
121.14% 확률로 긍정 리뷰입니다.



In [182]:
sentiment_predict('인테리어가 잘되어있고 케이크 맛있음')

['인테리어/Noun', '가/Josa', '자다/Verb', '되어다/Verb', '있다/Adjective', '케이크/Noun', '맛있다/Adjective']
[[58, 46, 148, 6, 30, 4]]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0  58  46 148   6  30   4]]
174.94% 확률로 긍정 리뷰입니다.



In [165]:
sentiment_predict('ㅋㅋㅋ별로')

['ㅋㅋㅋ/KoreanParticle', '별로/Noun']
[[34]]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0 34]]
6.40% 확률로 긍정 리뷰입니다.



In [166]:
sentiment_predict('애매해요')

['애매하다/Adjective']
[[1756]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0 1756]]
98.54% 확률로 긍정 리뷰입니다.

